# Drowsiness Detection

[Prashant Brahmbhatt](https://www/github.com/hashbanger)

*The following code is sugested to be written combinely in a python script as it requires command line arguments*

___

#### The procedure
The process is fairly straightforward:    
- The camera will detect the face.  
- Then we will extract the eye region from the facial landmark detection
- We will compute the eye aspect ratio to determine if the eyes are closed.
- If eyes remain closed for sufficiently long time will trigger an alarm.

In [2]:
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
import numpy as np
import playsound
import argparse
import imutils
import time
import dlib
import cv2

For mac OS one would also require `pip install pyobj`

First we define a function to play a sound.

In [4]:
def sound_alarm(path):
    '''Takes in the path of the sound to play, mp3/WAV'''
    playsound.playsound(path)

We will use our EAR calculation function more on that [here](https://github.com/hashbanger/Computer_Vision/blob/master/DrowsinessDetection/Eye_Blinking_Detection/py_cv_eyeBlinking.ipynb)

In [8]:
def eye_aspect_ratio(eye):
    '''Taking the array of eye coordinates and returning the ratio'''
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    
    ear = (A + B)/ (2 * C)
    
    return ear

Now in our drowsiness detection unlike the blinking case the eyes will not open but rather will remain closed. So the EAR value dropped will not increase again indicating the person has closed their eyes.

In [ ]:
ap = argparse.ArgumentParser()
ap.add_argument("-p", "--shape-predictor", required=True,
    help="path to facial landmark predictor")
ap.add_argument("-a", "--alarm", type=str, default="",
    help="path alarm .WAV file")
ap.add_argument("-w", "--webcam", type=int, default=0,
    help="index of webcam on system")
args = vars(ap.parse_args())

- `--shape-predictor` : The path for our pretrained facial landmark detector.
- `--alarm` : An optional argument to define the sound file to be played as alarm.
- `--webcam` : The default value is 0 as for in-built webcam, for external pass on 1.

The next step is to set up two constants that one may require tuning as per their requirements. We also require two variables declaration.

- The first constant is for setting the default threshold value.
- The second constant is the number of frames the EAR should be below threshold to set off the alarm.

In [10]:
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 48  # decreasing it will cause the detector to be more sensitive
 
# initialize the frame counter as well as a boolean used to
# indicate if the alarm is going off

COUNTER = 0
ALARM_ON = False

Now we will use the dlib's face detector(the pretrained model) to get the face in the frame

In [ ]:
print("Loading up the facial landmark detector")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(args["shape_predictor"])

As in the eye blink detection we get our eye indexes.

In [12]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [ ]:
print("setting up Video Stream...")
vs = VideoStream(src=args["webcam"]).start()
time.sleep(1.0)
 
# loop over frames from the video stream
while True:
    # grab the frame from the threaded video file stream, resize
    # it, and convert it to grayscale

    frame = vs.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 
    # detect faces in the grayscale frame
    rects = detector(gray, 0)
    
    for rect in rects:
        # determining the facial landmarks and then coverting them to a numpy array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # extract the left and right eye coordinates, then use the
        # coordinates to compute the eye aspect ratio for both eyes
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]

        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        # averaging the eye aspect ratio together for both eyes
        ear = (leftEAR + rightEAR) / 2.0
    
        # compute the convex hull for the left and right eye, then
        # visualize each of the eyes
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

        if ear < EYE_AR_THRESH:
            COUNTER += 1

            # if the eyes were close for sufficient time then sound the alarm
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                # if teh alarm is not turn it on
                if not ALARM_ON:
                    ALARM_ON = True

                # checking if the alarm file is supplied 
                # if present then we will play in the background using the Thread
                    if args['alarm'] != '':
                        t = Thread(target=sound_alarm, args= (args['alarm'],))
                        t.deamon = True
                        t.start()

                cv2.putText(frame, "DROWSINESS ALERT!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        else:
            COUNTER = 0
            ALARM_ON = False
        
        # draw the computed eye aspect ratio on the frame to help
        # with debugging and setting the correct eye aspect ratio
        # thresholds and frame counters
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
 
    # show the frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
 
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
#do a clean up
cv2.destroyAllWindows()
vs.stop()